In [1]:
pipelines


{'__default__': Pipeline([
Node(standardize_m3, 'm3_stock_dataset', 'm3_stock_parquet', 'standardize_m3'),
Node(standardize_reflex, 'reflex_stock_dataset', 'reflex_stock_parquet', 'standardize_reflex'),
Node(map_m3, {'m3_df': 'm3_stock_parquet', 'rules': 'params:m3_mapping_rules', 'pos_df': 'm3_po_dataset'}, 'm3_map', 'map_m3'),
Node(map_reflex, {'reflex_df': 'reflex_stock_parquet', 'mapping': 'params:reflex_mapping_rules'}, 'reflex_map', 'map_reflex_category'),
Node(build_reflex_m3_wide_with_lotless, {'reflex_map': 'reflex_map', 'm3_map': 'm3_map', 'depots': 'params:m3_depots_columns'}, 'corr_dataset', 'build_reflex_m3_wide'),
Node(compute_m3_reliquat, {'m3_map': 'm3_map', 'reflex_map': 'reflex_map'}, 'm3_reliquat', 'compute_m3_reliquat_node'),
Node(compute_m3_regul, 'corr_dataset', 'reflex_m3_regul', 'compute_m3_regul'),
Node(generate_api_m3_rfx, {'reflex_m3_regul': 'reflex_m3_regul', 'm3_map': 'm3_map'}, 'stock_m3_rfx', 'generate_stock_m3_rfx_node')
]), 'preprocessing': Pipeline([


In [2]:
catalog


'm3_stock_dataset': kedro_datasets.pandas.sql_dataset.SQLQueryDataset
'm3_po_dataset': kedro_datasets.pandas.sql_dataset.SQLQueryDataset
'reflex_stock_dataset': kedro_datasets.pandas.sql_dataset.SQLQueryDataset
'm3_stock_parquet': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'reflex_stock_parquet': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'm3_map': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'reflex_map': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'm3_reliquat': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'corr_dataset': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'reflex_m3_regul': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'stock_m3_rfx': kedro_datasets.pandas.csv_dataset.CSVDataset
'parameters': kedro.io.memory_dataset.MemoryDataset(data='<dict>')
'params:reflex_mapping_rules': kedro.io.memory_dataset.MemoryDataset(data='<dict>')
'params:reflex_mapping_rules.STD': kedro.io.memory_dataset.MemoryDataset(data='<str>')


## Analyse des cas 

### Cas listés 
https://wolfsa.sharepoint.com/sites/ERP/Documents%20partages/General/20%20Implementation/40%20Go%20live/Stocks-WMS/STOCK_251208/Base%20de%20travail%20stocks%20RFX%20vs%20M3%20NEW_251108.xlsx?web=1

In [3]:
m3 = catalog.load('m3_map')
reflex = catalog.load('reflex_map')
depots = ["100", "150", "200", "400"]

                    INFO     Loading data from m3_map (ParquetDataset)...                      ]8;id=246720;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=341152;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

                    INFO     Loading data from reflex_map (ParquetDataset)...                  ]8;id=623985;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=768686;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

In [4]:
reflex_sr = reflex[reflex.category != "RETOUR"]
m3_sr = m3[m3.category != "RETOUR"]

In [5]:
reflex_lot = reflex_sr[reflex_sr['lot'].notna()].copy()
m3_lot = m3_sr[m3_sr['lot'].notna()].copy

m3_groups = m3.groupby(["depot","lot", "category", "type", "sku"], dropna=False)["qty_m3"].sum().reset_index()
m3_pivot = m3_groups.pivot_table(
            index=["lot", "type", "sku", "category"],
            columns=["depot"],
            values="qty_m3",
            aggfunc="sum",
            fill_value=0,
        ).reset_index()
m3_pivot = m3_pivot.rename(
        columns={d: f"stock_{d}" for d in depots if d in m3_pivot.columns}
    )

In [6]:
wide_with_lot = reflex_lot.merge(
    m3_pivot,
    on=["category", "lot", "sku"],
    how="left",
)

In [7]:
wide_with_lot

,sku,lot,qualite,qty_reflex,category,type,stock_100,stock_150,stock_200,stock_400
0,BD60XAK840005,WF24-0057,STD,359.0,STOCK,A01,359.0,0.0,0.0,0.0
1,60PAN77JQZ4244,1000002553,STD,5.0,STOCK,A01,9.0,0.0,0.0,0.0
2,SC60PAF320008,WF20-0243,DES,1.0,DES,A01,0.0,0.0,1.0,0.0
3,SC70A921200W2,WF24-0133A,RET,2.0,NDISP,A01,2.0,0.0,0.0,0.0
4,SC70AAH62004D,WF23-0260A,STD,3.0,STOCK,A01,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
21367,SC70A56400AG,WF22-0073,STD,1.0,STOCK,A01,1.0,0.0,0.0,0.0
21368,70GAN69JTS080C,1000002567,STD,185.0,STOCK,A01,155.0,30.0,0.0,0.0
21369,70564JTC080D,1000002830,STD,447.0,STOCK,A01,449.0,0.0,0.0,0.0
21370,SC78AAK54000I,WF23-0247,STD,92.0,STOCK,A01,82.0,10.0,0.0,0.0


In [8]:
reflex_lotless = reflex[reflex['lot'].isna()].copy()
lotless_m3 = m3[m3['lot'].isna()].copy()

lotless_m3_groups = lotless_m3.groupby(["depot", "category", "type", "sku"], dropna=False)["qty_m3"].sum().reset_index()
lotless_m3_pivot = lotless_m3_groups.pivot_table(
            index=["type", "sku", "category"],
            columns=["depot"],
            values="qty_m3",
            aggfunc="sum",
            fill_value=0,
        ).reset_index()

In [9]:
wide_no_lot = reflex_lotless.merge(
    lotless_m3_pivot,
    on=["category", "sku"],
    how="left",
)

In [10]:
wide_no_lot

,sku,lot,qualite,qty_reflex,category,type,100,150
0,SCDIVERS003X,None,STD,17.0,STOCK,B01,17.0,0.0
1,CAT000000000036,None,STD,108.0,STOCK,B01,108.0,0.0
2,CAT000000000028,None,STD,115.0,STOCK,B01,115.0,0.0
3,DIV000000000456,None,STD,75.0,STOCK,B01,75.0,0.0
4,SCPREPAETIQ00AH,None,STD,187.0,STOCK,B02,187.0,0.0
...,...,...,...,...,...,...,...,...
252,PLMETI000008050,None,STD,250.0,STOCK,B02,250.0,0.0
253,BDDISPLAY002G,None,STD,4.0,STOCK,B01,4.0,0.0
254,PLMBOI000007936,None,STD,3977.0,STOCK,B02,3977.0,0.0
255,SCPREPAETIQ0055,None,STD,4360.0,STOCK,B02,4360.0,0.0


Reste à faire

### Gestion des Lots existants dans M3 mais absents de Reflex

Si le lot est absent dans Reflex, il ne redescendra pas dans les tables précédentes. 

In [11]:
lot_list = []

for i in m3_sr.lot.unique():
    if i not in reflex_sr.lot.unique():
        lot_list.append(i)

In [12]:
print(lot_list)

['OI0500093635', 'OI0500042023', 'OI0500073223', 'OI0500073224', 'OI0500042021', 'OI0500066358', 'OI0500066054', 'OI0500062460', 'OI0500098814', 'WF24-0006', 'OI0500096782', '1000002523', 'OI0500076766', 'OI0500059332', 'WF24-0008', 'OI0500104645', 'OI0500055378', 'OI0500104599', 'OI0500057424', 'OI0500058258', 'OI0500040694', 'OI0500073232', 'OI0500066905', 'OI0500042009', '1000002077', 'OI0500095338', 'OI0500095320', 'OI0500072331', 'OI0500073234', 'OI0500073247', 'OI0500038632', 'WF23-0167', '99999', 'OI0500076787', 'OI0500044598', 'OI0500098568', 'OI0500101092', '1000002376', 'OI0500067332', 'OI0500039049', 'OI0500097972', 'OI0500059341', 'OI0500059348', 'OI0500104646', 'OI0500042005', 'OI0500073243', '1000003093', 'OI0500072337', 'WF24-0146', 'WF23-0161', 'OI0500094798', 'OI0500072453', 'OI0500073237', 'OI0500095334', 'OI0500059533', 'OI0500061240', 'OI0500057537', 'OI0500076762', 'OI0500059188', 'OI0500072476', 'WF19-0029', 'OI0500076265', 'OI0500102657', 'OI0500099685', '1000002

In [13]:
m3[m3.lot == "OI0500093635"]

,sku,sku_m3,lot,depot,category,type,qty_m3,is_sms,is_150
68,SC60P1537000C,153744648,OI0500093635,100,RETOUR,A01,2.0,0,0
72,SC60P1537000C,153744648,OI0500093635,150,STOCK,A01,7.0,0,0
21848,SC79SAL460002,79SAL463085F,OI0500093635,100,RETOUR,A01,2.0,0,0


In [21]:
m3[m3.sku == "SC60P1537000C"]

,sku,sku_m3,lot,depot,category,type,qty_m3,is_sms,is_150
68,SC60P1537000C,153744648,OI0500093635,100,RETOUR,A01,2.0,0,0
69,SC60P1537000C,153744648,1000002661,100,STOCK,A01,315.0,0,0
70,SC60P1537000C,153744648,1000003060,100,STOCK,A01,790.0,0,0
71,SC60P1537000C,153744648,1000002661,150,STOCK,A01,6.0,0,0
72,SC60P1537000C,153744648,OI0500093635,150,STOCK,A01,7.0,0,0
163,SC60P1537000C,153744648,1000002784,100,STOCK,A01,960.0,0,0
164,SC60P1537000C,153744648,1000002422,150,STOCK,A01,2.0,0,0


In [15]:
reflex[reflex.sku == "SC60P1537000C"]

,sku,lot,qualite,qty_reflex,category
6025,SC60P1537000C,1000003060,STD,790.0,STOCK
6544,SC60P1537000C,1000002661,STD,335.0,STOCK
12683,SC60P1537000C,1000002784,STD,960.0,STOCK
16962,SC60P1537000C,1000002422,STD,1.0,STOCK


In [16]:
wide_with_lot[wide_with_lot.lot == "OI0500093635"]

,sku,lot,qualite,qty_reflex,category,type,stock_100,stock_150,stock_200,stock_400


In [17]:
df = catalog.load('corr_dataset')

[12/18/25 15:47:59] INFO     Loading data from corr_dataset (ParquetDataset)...                ]8;id=156395;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=890166;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

In [18]:
df[df['ecart_rfx_m3'] != 0]

,sku,lot,qualite,type,category,qty_reflex,stock_100,stock_200,stock_150,stock_400,stock_total_m3,ecart_rfx_m3
1,60PAN77JQZ4244,1000002553,STD,A01,STOCK,5.0,9.0,0.0,0.0,0.0,9.0,-4.0
8,20SAN98HRU000MS,1000002464,STD,A06,STOCK,6.0,0.0,0.0,0.0,7.0,7.0,-1.0
11,SC70A564008I,1000002606,STD,A01,STOCK,32.0,16.0,0.0,12.0,0.0,28.0,4.0
15,SC40PAL410009,WF24-0138,STD,A01,STOCK,209.0,189.0,0.0,10.0,0.0,199.0,10.0
19,SC70AAH610010,EAN-CHN,STD,A01,STOCK,2.0,1.0,0.0,2.0,0.0,3.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
21558,SC20AAA730006,None,None,None,STOCK,1.0,0.0,0.0,0.0,0.0,0.0,1.0
21559,SC20AAA73000B,None,None,None,STOCK,1.0,0.0,0.0,0.0,0.0,0.0,1.0
21560,SC20AAA73000E,None,None,None,STOCK,2.0,0.0,0.0,0.0,0.0,0.0,2.0
21561,SCBOÎTE0006,None,None,None,STOCK,23.0,0.0,0.0,0.0,0.0,0.0,23.0


In [19]:
df[df.lot == "WF24-0133A"]

,sku,lot,qualite,type,category,qty_reflex,stock_100,stock_200,stock_150,stock_400,stock_total_m3,ecart_rfx_m3
3,SC70A921200W2,WF24-0133A,RET,A01,NDISP,2.0,2.0,0.0,0.0,0.0,2.0,0.0
727,SC70A921200VG,WF24-0133A,RET,A01,NDISP,6.0,1.0,0.0,0.0,0.0,1.0,5.0
983,SC70A921200VS,WF24-0133A,RET,A01,NDISP,3.0,2.0,0.0,0.0,0.0,2.0,1.0
2473,SC70A921200V7,WF24-0133A,STD,A01,STOCK,46.0,27.0,0.0,15.0,0.0,42.0,4.0
2695,SC70A921200W3,WF24-0133A,RET,A01,NDISP,1.0,2.0,0.0,0.0,0.0,2.0,-1.0
3398,SC70A921200VH,WF24-0133A,STD,A01,STOCK,17.0,15.0,0.0,3.0,0.0,18.0,-1.0
3782,SC70A921200VG,WF24-0133A,STD,A01,STOCK,31.0,21.0,0.0,10.0,0.0,31.0,0.0
6971,SC70A921200W0,WF24-0133A,STD,A01,STOCK,1.0,0.0,0.0,1.0,0.0,1.0,0.0
7202,SC70A921200VI,WF24-0133A,STD,A01,STOCK,41.0,27.0,0.0,15.0,0.0,42.0,-1.0
7576,SC70A921200W1,WF24-0133A,STD,A01,STOCK,2.0,2.0,0.0,0.0,0.0,2.0,0.0


In [20]:
m3[m3.sku == "SC70A921200W2"]

,sku,sku_m3,lot,depot,category,type,qty_m3,is_sms,is_150
12194,SC70A921200W2,709212JIO080D,WF24-0100A,100,STOCK,A01,2.0,0,0
12195,SC70A921200W2,709212JIO080D,WF24-0100A,150,STOCK,A01,1.0,0,0
12342,SC70A921200W2,709212JIO080D,WF24-0133A,100,NDISP,A01,2.0,0,0
12343,SC70A921200W2,709212JIO080D,WF24-0038,100,STOCK,A01,2.0,0,0
12344,SC70A921200W2,709212JIO080D,WF24-0133A,150,STOCK,A01,13.0,0,0
13875,SC70A921200W2,709212JIO080D,WF24-0133A,100,STOCK,A01,102.0,0,0
25478,SC70A921200W2,709212JIO080D,WF24-0038,150,STOCK,A01,2.0,0,0
